#  Clinton and Trump election Tweets Analysis


Launay Christian & Alfred Chantharath

20/10/2018

* 1 Introduction
* 2 Data Source
* 3 Preparation
* 4 Data Content: First 5 Rows
* 4.1 Overall Tweets and Retweets view 
* 5 Time Analysis 
* 6 Text Mining
* 6.1 Sentiment Analysis
* 7 Conclusion

# 1. Introduction

the 2016 US Presidential Election is one of the most publicized in the history and Twitter has played an increasingly prominent role in it. Debates have raged and candidates have risen and fallen based on tweets.

So we finded interesting to analyse the tweets tendency per device and the sentiments communicate in them .

# 2. Data Source 

we used the Hillary Clinton and Donald Trump Tweets dataset which contain ~3000 recent tweets from Hillary Clinton and Donald Trump, the two major-party presidential nominees.



# 3. Preparation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Input data file
tweets = pd.read_csv('../input/tweets.csv')


In [ ]:
#Separating out the time variable by Hour, Day, Month and Year 
#for further analysis using datetime package
import datetime as dt
tweets['time'] = pd.to_datetime(tweets['time'])
tweets['hour'] = tweets['time'].apply(lambda x: x.hour)
tweets['month'] = tweets['time'].apply(lambda x: x.month)
tweets['day'] = tweets['time'].apply(lambda x: x.day)
tweets['year'] = tweets['time'].apply(lambda x: x.year)


# 4.Data Content: First 5 Rows¶
 

In [ ]:
tweets.head(5)

# 4.1 Overall Tweets and Retweets visualization 

In [ ]:
#Total number of tweets by both of the twitter handles
sns.countplot(x='handle', data = tweets)

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,3))

#Total number of original tweets and retweets for each of the contender
retweet_hc = tweets.loc[(tweets['handle']=='HillaryClinton'), ['is_retweet']]
retweet_dt = tweets.loc[(tweets['handle']=='realDonaldTrump'), ['is_retweet']]
ax1 = sns.countplot(retweet_hc['is_retweet'], palette='rainbow', ax=axis1)
ax1.set_title("HillaryClinton's tweets")
ax1.set(xticklabels=["Tweets","Retweets"])

ax2 = sns.countplot(retweet_dt['is_retweet'], palette="Set1", ax=axis2)
ax2.set_title("realDonaldTrump's tweets")
ax2.set(xticklabels=["Tweets","Retweets"])

from this we can say that Donald Trump has used a little a bit more twitter than Hillary Clinton but Clinton's tweets have been more retweeted than Trump's one 

# 5. Time Analysis of the number of tweets  

In [ ]:
#Number of tweets by the months
monthly_tweets = tweets.groupby(['month', 'handle']).size().unstack()
monthly_tweets.plot(title='Monthly Tweet Counts', colormap='winter')

In [ ]:
#Number of tweets daily
daily_tweets = tweets.groupby(['day', 'handle']).size().unstack()
daily_tweets.plot(title='Daily Tweet Counts')

In [ ]:
#Number of tweets hourly
hourly_tweets = tweets.groupby(['hour', 'handle']).size().unstack()
hourly_tweets.plot(title='Hourly Tweet Counts', colormap='coolwarm')

Hillary Clinton tweets start in april meanwhile Donald Trump has started in january that's why he have more tweets than Clinton's . Moreover it's interesting to see the hourly tweets tendency , canditates tend to communicate more in the afternoon and the night .

# 6. Text mining 



In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS

tweets_hillary=tweets.loc[(tweets['handle']=='HillaryClinton'),['text']]
tweets_trump=tweets.loc[(tweets['handle']=='realDonaldTrump'),['text']]

In [ ]:
stopwords = set(STOPWORDS)
stopwords.add("http")
stopwords.add("https")
stopwords.add("amp")
stopwords.add("CO")
stopwords.add("Trump")
stopwords.add("Trump2016")
stopwords.add("Donald")
stopwords.add("Clinton")
stopwords.add("Hillary")
stopwords.add("realDonaldTrump")
stopwords.add("will")
stopwords.add("say")
stopwords.add("said")
stopwords.add("let")
stopwords.add("vote")
stopwords.add("now")
stopwords.add("go")

# The words Clinton used the most

In [ ]:
wordcloud_hc = WordCloud(background_color='white',max_font_size=46, relative_scaling=0.5,stopwords=stopwords).generate(tweets_hillary['text'].str.cat())
plt.figure()
plt.imshow(wordcloud_hc)
plt.axis("off")
plt.show()

# The words  Trump used the most

In [ ]:
wordcloud_dt = WordCloud(max_font_size=42, relative_scaling=.5,stopwords=stopwords).generate(tweets_trump['text'].str.cat())
plt.figure()
plt.imshow(wordcloud_dt)
plt.axis("off")
plt.show()

# 6.1 Sentiment Analysis

# Categorize the text column into Positive and Negative sentiments using TextBlob



The TextBlop package contains several sentiment lexicons in the sentiments dataset.




# Sentiment Analysis on the entiere dataset 

In [ ]:
from textblob import TextBlob

bloblist_desc = list()

df_tweet_descr_str=tweets['text'].astype(str)
for row in df_tweet_descr_str:
    blob = TextBlob(row)
    bloblist_desc.append((row,blob.sentiment.polarity, blob.sentiment.subjectivity))
    df_tweet_polarity_desc = pd.DataFrame(bloblist_desc, columns = ['sentence','sentiment','polarity'])
 
def f(df_tweet_polarity_desc):
    if df_tweet_polarity_desc['sentiment'] > 0:
        val = "Positive"
    elif df_tweet_polarity_desc['sentiment'] == 0:
        val = "Neutral"
    else:
        val = "Negative"
    return val

df_tweet_polarity_desc['Sentiment_Type'] = df_tweet_polarity_desc.apply(f, axis=1)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.countplot(x="Sentiment_Type", data=df_tweet_polarity_desc)

Unlike what we could have think, most of the tweets are postive and neutral. we know are going to focus on each candidates .

# Clinton Sentiment analysis

In [ ]:
tweet_clinton = tweets.loc[(tweets['handle']=='HillaryClinton'), ['text']]
bloblist_desc = list()

df_tweet_clinton_str=tweet_clinton['text'].astype(str)
for row in df_tweet_clinton_str:
    blob = TextBlob(row)
    bloblist_desc.append((row,blob.sentiment.polarity, blob.sentiment.subjectivity))
    df_tweet_clinton_polarity_desc = pd.DataFrame(bloblist_desc, columns = ['sentence','sentiment','polarity'])
 
def f(df_tweet_clinton_polarity_desc):
    if df_tweet_clinton_polarity_desc['sentiment'] > 0:
        val = "Positive"
    elif df_tweet_clinton_polarity_desc['sentiment'] == 0:
        val = "Neutral"
    else:
        val = "Negative"
    return val

df_tweet_clinton_polarity_desc['Sentiment_Type'] = df_tweet_clinton_polarity_desc.apply(f, axis=1)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.countplot(x="Sentiment_Type", data=df_tweet_clinton_polarity_desc)
          


It's interesting to see that Hillary Clinton has a large number of neutral tweets , it maybe was a choice for her campain . 

# Trump Sentiment Analysis

In [ ]:
tweet_trump = tweets.loc[(tweets['handle']=='realDonaldTrump'), ['text']]
bloblist_desc = list()

df_tweet_trump_str=tweet_trump['text'].astype(str)
for row in df_tweet_trump_str:
    blob = TextBlob(row)
    bloblist_desc.append((row,blob.sentiment.polarity, blob.sentiment.subjectivity))
    df_tweet_trump_polarity_desc = pd.DataFrame(bloblist_desc, columns = ['sentence','sentiment','polarity'])
 
def f(df_tweet_trump_polarity_desc):
    if df_tweet_trump_polarity_desc['sentiment'] > 0:
        val = "Positive"
    elif df_tweet_trump_polarity_desc['sentiment'] == 0:
        val = "Neutral"
    else:
        val = "Negative"
    return val

df_tweet_trump_polarity_desc['Sentiment_Type'] = df_tweet_trump_polarity_desc.apply(f, axis=1)

plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.countplot(x="Sentiment_Type", data=df_tweet_trump_polarity_desc)

It's really interesting to see that Donald Trump has a really large number of positive tweets , their represent nearly the double of his neutral or negative tweets 

# 7. Conclusion

sentiment analysis can be really usefull to break down a comportement, a tendency that would not have been an intuitive one at the first insight . this was our first text analysis we will continue this work in the following weeks 